# BQSKit Demo for AIDE-QC All Hands

This demo is performed with:
- Python 3.9.3
- QSearch 2.5.0
- QFAST 2.2.0
- QFactor 1.0.1

Can be installed by pip: `pip install qsearch qfast qfactor`

## General Synthesis with QSearch + LEAP

In [ ]:
from qsearch import options, leap_compiler, compiler, post_processing, assemblers, unitaries

# Pass options into qsearch and set the target to utry
opts = options.Options()
opts.target = unitaries.qft(8)  # Synthesize 3-qubit unitary
opts.verbosity = 0              # Don't log
opts.write_to_stdout = False    # Don't print log messages to stdout
opts.reoptimize_size = 7        # Reoptimization window

# Use the LEAP compiler
compiler = leap_compiler.LeapCompiler()
output = compiler.compile(opts)

# LEAP's Reoptimization as QSearch post-processing
pp = post_processing.LEAPReoptimizing_PostProcessor()
output = pp.post_process_circuit(output, opts)
qasm_str = assemblers.ASSEMBLER_IBMOPENQASM.assemble(output)
print(qasm_str)

### Changing topology

In [ ]:
from qsearch import gatesets

# Set to topology to all-to-all, qsearch defaults to linear.
opts.gateset = gatesets.QubitCNOTAdjacencyList([(0,1),(0,2),(1,2)])

# Recompile and reoptimize
compiler = leap_compiler.LeapCompiler()
output = compiler.compile(opts)
pp = post_processing.LEAPReoptimizing_PostProcessor()
output = pp.post_process_circuit(output, opts)
qasm_str = assemblers.ASSEMBLER_IBMOPENQASM.assemble(output)
print(qasm_str)

### Different Gatesets

In [ ]:
# Set gateset to different one
# opts.gateset = gatesets.QubitCZLinear()  # CZ Gates
opts.gateset = gatesets.QubitXXLinear()  # XX Gates
# opts.gateset = gatesets.QubitISwapLinear()  # ISwap Gates

# Recompile and reoptimize
compiler = leap_compiler.LeapCompiler()
output = compiler.compile(opts)
pp = post_processing.LEAPReoptimizing_PostProcessor()
output = pp.post_process_circuit(output, opts)
qasm_str = assemblers.ASSEMBLER_IBMOPENQASM.assemble(output)
print(qasm_str)

### Optimizer Settings

In [ ]:
from qsearch import solvers, multistart_solvers, parallelizers

# Reoptimization window
opts.reoptimize_size = 5

# Multistart Solver
opts.solver = multistart_solvers.MultiStart_Solver(8)
opts.parallelizer = parallelizers.ProcessPoolParallelizer

# Recompile and reoptimize
compiler = leap_compiler.LeapCompiler()
output = compiler.compile(opts)
pp = post_processing.LEAPReoptimizing_PostProcessor()
output = pp.post_process_circuit(output, opts)
qasm_str = assemblers.ASSEMBLER_IBMOPENQASM.assemble(output)
print(qasm_str)

### With logging

In [ ]:
opts.verbosity = 2
opts.write_to_stdout = True


# Recompile and reoptimize
compiler = leap_compiler.LeapCompiler()
output = compiler.compile(opts)
pp = post_processing.LEAPReoptimizing_PostProcessor()
output = pp.post_process_circuit(output, opts)
qasm_str = assemblers.ASSEMBLER_IBMOPENQASM.assemble(output)
print(qasm_str)

## General Synthesis with QFAST

In [ ]:
import qfast

qasm_str = qfast.synthesize(unitaries.qft(8))  # Synthesize qft3
print(qasm_str)

### Changing topology

In [ ]:
# Set to linear topology, defaults to all-to-all
qasm_str = qfast.synthesize(unitaries.qft(8), coupling_graph = [(0,1),(1,2)])
print(qasm_str)

### Different Gatesets

In [ ]:
# Set to cz gates, defaults to cx, can also take iswap and rxx
qasm_str = qfast.synthesize(unitaries.qft(8), basis_gates = ['cz'])
print(qasm_str)

### Optimizer Settings

In [ ]:
# Set success threshold
qasm_str = qfast.synthesize(unitaries.qft(8), model_options = {"success_threshold": 1e-6})
print(qasm_str)

### Block size settings

In [ ]:
# Use different block sizes
# Default hierarchy fn = lambda x : x // 3 if x > 5 else 2
qasm_str = qfast.synthesize(unitaries.qft(16), hierarchy_fn = lambda x : 3)  # 3-qubit blocks
print(qasm_str)

### With logging

In [ ]:
# Enable qfast logging
import logging
logging.getLogger( "qfast" ).setLevel( logging.INFO )

qasm_str = qfast.synthesize(unitaries.qft(16))  # Synthesize qft4
print(qasm_str)

## Circuit Template Instantiation with QFactor

In [ ]:
"""Optimize a 3-qubit circuit to be a toffoli gate."""

import numpy as np
from scipy.stats import unitary_group

from qfactor import Gate, optimize, get_distance


# The next two lines start qfactor's logger.
import logging
logging.getLogger( "qfactor" ).setLevel( logging.INFO )

# We will optimize towards the toffoli unitary.
toffoli = np.array( [ [ 1, 0, 0, 0, 0, 0, 0, 0 ],
                      [ 0, 1, 0, 0, 0, 0, 0, 0 ],
                      [ 0, 0, 1, 0, 0, 0, 0, 0 ],
                      [ 0, 0, 0, 1, 0, 0, 0, 0 ],
                      [ 0, 0, 0, 0, 1, 0, 0, 0 ],
                      [ 0, 0, 0, 0, 0, 1, 0, 0 ],
                      [ 0, 0, 0, 0, 0, 0, 0, 1 ],
                      [ 0, 0, 0, 0, 0, 0, 1, 0 ] ] )

# Start with the circuit structure
# and an initial guess for the gate's unitaries.
# Here we use randomly generated unitaries for initial guess.
circuit = [ Gate( unitary_group.rvs(4), (1, 2) ),
            Gate( unitary_group.rvs(4), (0, 2) ),
            Gate( unitary_group.rvs(4), (1, 2) ),
            Gate( unitary_group.rvs(4), (0, 2) ),
            Gate( unitary_group.rvs(4), (0, 1) ) ]

# Note: the Gate object also has an optional boolean parameter "fixed"
# If "fixed" is set to true, that gate's unitary will not change.

# Call the optimize function
ans = optimize( circuit, toffoli, # <--- These are the only required args
                diff_tol_a = 1e-12,   # Stopping criteria for distance change
                diff_tol_r = 1e-6,    # Relative criteria for distance change
                dist_tol = 1e-12,     # Stopping criteria for distance
                max_iters = 100000,   # Maximum number of iterations
                min_iters = 1000,     # Minimum number of iterations
                slowdown_factor = 0 ) # Larger numbers slowdown optimization
                                      # to avoid local minima


# The result "ans" is another circuit object (list[Gate])
# with the gate's unitaries changed from the input circuit.
print( "Circuit: ", ans )
print( "Final Distance: ", get_distance( ans, toffoli ) )